In [ ]:
# Importing necessary libraries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [ ]:
# Load and preprocess the dataset
max_features = 10000  # Number of words to consider as features
maxlen = 100  # Cut texts after this number of words (among top max_features most common words)

In [ ]:
# (a) Preprocessing of the Data
print("Loading data...")
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

print(f"Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

Loading data...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Pad sequences (samples x time)


In [ ]:
# (b) Divide data into training and testing data set
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (25000, 100)
Testing data shape: (25000, 100)


In [ ]:
# (c) Build the GRU Model
def build_gru_model():
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(GRU(128, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# Build the LSTM Model for comparison
def build_lstm_model():
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
# (d) Training the GRU Model
gru_model = build_gru_model()
print("Training GRU model...")
gru_model.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.2)

# Training the LSTM Model
lstm_model = build_lstm_model()
print("Training LSTM model...")
lstm_model.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.2)

Training GRU model...
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 109s 167ms/step - accuracy: 0.7149 - loss: 0.5313 - val_accuracy: 0.8546 - val_loss: 0.3447
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 100s 161ms/step - accuracy: 0.9009 - loss: 0.2505 - val_accuracy: 0.8588 - val_loss: 0.3348
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 98s 157ms/step - accuracy: 0.9447 - loss: 0.1520 - val_accuracy: 0.8456 - val_loss: 0.3677
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 154ms/step - accuracy: 0.9686 - loss: 0.0948 - val_accuracy: 0.8444 - val_loss: 0.4993
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 154ms/step - accuracy: 0.9855 - loss: 0.0461 - val_accuracy: 0.8382 - val_loss: 0.5980
Training LSTM model...
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 120s 188ms/step - accuracy: 0.7184 - loss: 0.5258 - val_accuracy: 0.8326 - val_loss: 0.3723
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 144s 192ms/step - accuracy: 0.8929 - loss: 0.2606 - val_accuracy: 0.8264 - val_loss: 0.3835
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 

In [9]:
# (e) Evaluate Models
gru_score, gru_acc = gru_model.evaluate(X_test, y_test)
lstm_score, lstm_acc = lstm_model.evaluate(X_test, y_test)

print(f"GRU Model Accuracy: {gru_acc * 100:.2f}%")
print(f"LSTM Model Accuracy: {lstm_acc * 100:.2f}%")

782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.8359 - loss: 0.6375
782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 64ms/step - accuracy: 0.8261 - loss: 0.5926
GRU Model Accuracy: 83.65%
LSTM Model Accuracy: 82.90%


In [10]:
# (f) Text Generation (Simplified Example)

def generate_text(model, start_text, word_index, index_word, max_len=100):
    for _ in range(max_len):
        encoded_input = np.array([word_index.get(word, 0) for word in start_text])
        encoded_input = sequence.pad_sequences([encoded_input], maxlen=maxlen)
        prediction = model.predict(encoded_input)
        next_word = index_word.get(np.argmax(prediction), '?')
        start_text += ' ' + next_word
        if next_word == 'end':
            break
    return start_text

In [11]:
word_index = imdb.get_word_index()
index_word = {index: word for word, index in word_index.items()}

start_text = "the movie was"
generated_text_gru = generate_text(gru_model, start_text, word_index, index_word)
generated_text_lstm = generate_text(lstm_model, start_text, word_index, index_word)

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [12]:
print("Text Generated by GRU Model:", generated_text_gru)
print("Text Generated by LSTM Model:", generated_text_lstm)

Text Generated by GRU Model: the movie was ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Text Generated by LSTM Model: the movie was ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?


In [13]:
lstm_score, lstm_acc = lstm_model.evaluate(X_test, y_test)
gru_score, gru_acc = gru_model.evaluate(X_test, y_test)

# Print the accuracy results
print(f"LSTM Model Accuracy: {lstm_acc * 100:.2f}%")
print(f"GRU Model Accuracy: {gru_acc * 100:.2f}%")

782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 63ms/step - accuracy: 0.8261 - loss: 0.5926
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 34ms/step - accuracy: 0.8359 - loss: 0.6375
LSTM Model Accuracy: 82.90%
GRU Model Accuracy: 83.65%
